In [1]:
# filtered or not
# word2vec choice
# with images or not.
# pca


# main function is f(user, business)
# user2avgVec # inclusive of current business
# user2avgHistVec # exclusive of current business
# business2avgVec # inclusive of current user
# business2avgHistVec # exclusive of current user

# with or without t filtered json?
# with or without currents?
# with or without pics?

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import confusion_matrix
import seaborn as sns

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

import os
from collections import defaultdict
from bs4 import BeautifulSoup  
import requests
import zipfile
import io
import nltk
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
#from nltk.tokenize import word_tokenize
nltk.download('stopwords')

#import geopandas as gpd|
#import shapely
import gensim
import tqdm
import tensorflow as tf
import keras

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.manifold import SpectralEmbedding

# Load the Drive helper and mount
#from google.colab import drive
#mounted_path_folder = '/content/drive'
#drive.mount(mounted_path_folder, force_remount=True)

import sys
path_folder = "/content/drive/MyDrive/dsprojects/dsproject_grev/" # parent of current src folder
path_folder = (os.path.abspath(os.path.join((os.path.abspath(os.path.join(os.getcwd(), os.pardir))),os.pardir)))
sys.path.insert(0, path_folder+"/src/features/")
import util

from word2vec_recipe import *


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Benson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from keras.applications.vgg16 import VGG16
#vggnet_model = VGG16()
url_header_pics = "https://lh5.googleusercontent.com/p/"
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

from PIL import Image
import requests
from io import BytesIO

path_file_name = os.path.join(path_folder, "data", "temp","business_user2picVec.pkl")

import string

stop_words = stopwords.words('english')

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import tqdm

In [7]:
path_folder = "/content/drive/MyDrive/dsprojects/dsproject_grev/" # parent of current src folder
path_folder = (os.path.abspath(os.path.join((os.path.abspath(os.path.join(os.getcwd(), os.pardir))),os.pardir)))


In [8]:
import pickle

vocab_column_names = ["ingredients"]+["name"]
file_name_word2vec = "word2vec_"+("_".join(vocab_column_names))+".pkl"
path_file_word2vec = os.path.join(path_folder, "data","temp",file_name_word2vec)
with open(path_file_word2vec, "rb") as f:
    recipe_word2vec = pickle.load(f)

vocab_column_names = ["ingredients"]#+["name"]
file_name_tfidf = "tfidf_"+("_".join(vocab_column_names))+".pkl"
path_file_tfidf = os.path.join(path_folder, "data","temp",file_name_tfidf)
with open(path_file_tfidf, "rb") as f:
    recipe_tfidf = pickle.load(f)

DATASET_VERSION = ["FULL", "FILTER"][0]  
DROP_ONE_TIME_APPEARANCES = True
DROP_SEPARATED_ONE_TIME_APPEARANCES = False
USING_PICS = False

if DATASET_VERSION == "FILTER":
    path_folder_data = os.path.join(path_folder,"data","raw","filter_all_t.json")
else:
    path_folder_data = os.path.join(path_folder,"data","raw","image_review_all.json")

In [9]:
data = []
with open(path_folder_data) as f:
    for line in tqdm.tqdm(f):
        data.append(eval(line))
    f.close()

1487747it [01:32, 16117.05it/s]


In [10]:
# Filter or Full
# Pics or no Pics
# Drop one-time-appearing business_ids/user_ids: (seperately, or conjoined?) (for example, the business_id is 1-time, but the user_id has already appeared many times before)

In [11]:
if DATASET_VERSION == "FILTER":
    train_reviews = (data[0]["train"])
    test_reviews = (data[0]["test"])
else:
    train_reviews = data
# 3 tables:
# 1 for business_id, user_id, review_text, rating
# 1 for pics: business_id, user_id, pic
# 1 for external history review: business_id, user_id, review_text

#reviews_df = pd.DataFrame(data=None, columns=["business_id","user_id","rating","review_text"])
#history_reviews_df = pd.DataFrame(data=None, columns=["business_id","user_id","review_text"])
#pics_df = pd.DataFrame(data=None, columns=["business_id","user_id","pics"])

reviews_df_business_id_column_list = []
reviews_df_user_id_column_list = []
reviews_df_rating_column_list = []
reviews_df_review_text_column_list = []

history_reviews_df_business_id_column_list = []
history_reviews_df_user_id_column_list = []
history_reviews_df_review_text_column_list = []

pics_df_business_id_column_list = []
pics_df_user_id_column_list = []
pics_df_pics_column_list = []


for review in tqdm.tqdm(train_reviews):
    temp_business_id = None
    temp_user_id = None
    temp_rating = None
    temp_review_text = None
    temp_pics = None
    temp_history_reviews = None

    if "business_id" in review:
        temp_business_id = review["business_id"]
    if "user_id" in review:
        temp_user_id = review["user_id"]
    if "rating" in review:
        temp_rating = review["rating"]
    if "review_text" in review:
        temp_review_text = review["review_text"]
    if "pics" in review:
        temp_pics = review["pics"]
        if DATASET_VERSION == "FULL":
             temp_pics = [x["id"] for x in (temp_pics)]
    if "history_reviews" in review:
        temp_history_reviews = review["history_reviews"]
    
    reviews_df_business_id_column_list.append(temp_business_id)
    reviews_df_user_id_column_list.append(temp_user_id)
    reviews_df_rating_column_list.append(temp_rating)
    reviews_df_review_text_column_list.append(temp_review_text)

    if USING_PICS == True:
        if (temp_pics is None)==False:
            temp_pics_len = len(temp_pics)
            pics_df_business_id_column_list.extend([temp_business_id for _ in range(temp_pics_len)])
            pics_df_user_id_column_list.extend([temp_user_id for _ in range(temp_pics_len)])
            pics_df_pics_column_list.extend(temp_pics)

    #if (temp_history_reviews is None)==False:
    #    temp_history_reviews_len = len(temp_history_reviews)
    #    history_reviews_df_business_id_column_list.extend([x[0].split("_")[1] for x in temp_history_reviews])
    #    history_reviews_df_user_id_column_list.extend([temp_user_id for _ in range(temp_history_reviews_len)])
    #    history_reviews_df_review_text_column_list.extend([x[1] for x in temp_history_reviews])


reviews_df = pd.DataFrame()
reviews_df["business_id"] = reviews_df_business_id_column_list
reviews_df["user_id"] = reviews_df_user_id_column_list
reviews_df["rating"] = reviews_df_rating_column_list
reviews_df["review_text"] = reviews_df_review_text_column_list

#history_reviews_df = pd.DataFrame()
#history_reviews_df["business_id"] = history_reviews_df_business_id_column_list
#history_reviews_df["user_id"] = history_reviews_df_user_id_column_list
#history_reviews_df["review_text"] = history_reviews_df_review_text_column_list
if USING_PICS == True:
    pics_df = pd.DataFrame()
    pics_df["business_id"] = pics_df_business_id_column_list
    pics_df["user_id"] = pics_df_user_id_column_list
    pics_df["pics"] = pics_df_pics_column_list


100%|████████████████████████████████████████████████████████████████████| 1487747/1487747 [00:03<00:00, 440107.62it/s]


In [12]:
from sklearn.preprocessing import LabelEncoder

business_id_label_encoder = LabelEncoder()
user_id_label_encoder = LabelEncoder()

business_id_label_encoder.fit(reviews_df["business_id"])
user_id_label_encoder.fit(reviews_df["user_id"])

reviews_df["business_id"] = business_id_label_encoder.transform(reviews_df["business_id"])
reviews_df["user_id"] = user_id_label_encoder.transform(reviews_df["user_id"])

In [13]:
vocab = [word for word in recipe_word2vec.wv.vocab
         if ((word in recipe_tfidf.vocab2idx) and (word not in stop_words))
        ]
vecs = [recipe_word2vec.wv[word] for word in vocab]
vecs_arr = np.array(vecs)

word2vec_df = pd.DataFrame()
word2vec_df["review_text"] = vocab
#word2vec_df["vec"] = vecs
vec_column_names = ["v{0}".format(i) for i in range(vecs_arr.shape[1])]
for i in range(len(vec_column_names)):
    vec_column_name = vec_column_names[i]
    word2vec_df[vec_column_name] = vecs_arr[:, i]

In [14]:
non_alphanumeric = string.punctuation

stop_words = stopwords.words("english")
recipe_word2vec_lexicon = recipe_word2vec.wv.vocab
recipe_tfidf_lexicon = list(recipe_tfidf.vocab2idx.keys())

In [ ]:
reviews_df.to_csv("reviews.csv",index=False,  encoding='utf-32')

In [18]:
# pretend the filt was stacked with train test and pics words added already

In [33]:
reviews_df

,business_id,user_id,rating,review_text
0,64526,655093,4,We came for a birthday brunch and this place i...
1,64526,7725,5,"Cool place to hang out, have drinks. There is..."
2,64526,636015,2,This place doesn’t rock the senses when it com...
3,64526,530475,3,Pretty cool place to hang out with friends and...
4,64526,201693,5,We saw this place the other day when we had re...
...,...,...,...,...
1487742,0,366969,5,Ordered the Chicken Tikka Masala and Chicken K...
1487743,0,109700,5,Great shrimp tikka masala. A little too much r...
1487744,0,735645,4,"Good food, great portions. The chicken I order..."
1487745,0,62213,4,Amazing is really good you guys should try it ...


## **Pyspark**

In [19]:
import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.ml as M
from pyspark.sql.window import Window
from pyspark.sql import SparkSession

In [20]:
findspark.init()
#spark = SparkSession.builder.getOrCreate()

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "3g") \
    .appName('food_rec') \
    .getOrCreate()

In [21]:
# https://csyhuang.github.io/2020/09/17/split-vector-to-columns/
# https://stackoverflow.com/questions/47874037/convert-pyspark-dataframe-column-from-array-to-new-columns

In [30]:
reviews_df["review_text"].fillna("_",inplace=True)

In [ ]:
schema = T.StructType([
    T.StructField("business_id", T.IntegerType(), True),
    T.StructField("user_id", T.IntegerType(), True),
    T.StructField("rating", T.IntegerType(), True),
    T.StructField("review_text", T.StringType(), True)]
)
reviews_df = spark.createDataFrame(reviews_df, schema)

In [ ]:
schema = T.StructType([
    T.StructField("review_text", T.StringType(), True)
]+[T.StructField(c, T.DoubleType(), True) for c in vec_column_names]
)
word2vec_df = spark.createDataFrame(word2vec_df, schema)

In [16]:
temp_df = reviews_df

win_row_number = Window.orderBy("review_id")
temp_df = temp_df.withColumn('review_id',F.row_number().over(win_row_number))
id_df = temp_df.select(F.col("user_id"),F.col("business_id"),F.col("review_id"))
review_df = temp_df.select(F.col("review_id"),F.col("review_text"))

In [ ]:

review_df = review_df.withColumn("review_text",F.lower(col("review_text")))
review_df = review_df.withColumn("review_text", F.translate("review_text", non_alphanumeric, " "*len(non_alphanumeric)))
# https://sparkbyexamples.com/pyspark/pyspark-replace-column-values/
review_df = review_df.withColumn("review_text", F.split("review_text", " "))
# https://sparkbyexamples.com/pyspark/pyspark-convert-string-to-array-column/  

In [18]:
# reviewTextWords_df = review_df.explode("review_text")

reviewTextWords_df = review_df.withColumn("review_text", F.explode("review_text"))
# https://stackoverflow.com/questions/44436856/explode-array-data-into-rows-in-spark    

In [ ]:


reviewTextWords_df = reviewTextWords_df.filter(~((F.col("review_text")).isin(stop_words)))
reviewTextWords_df = reviewTextWords_df.filter(((F.col("review_text")).isin(recipe_word2vec_lexicon)))
reviewTextWords_df = reviewTextWords_df.filter(((F.col("review_text")).isin(recipe_tfidf_lexicon)))
reviewTextWords_df = reviewTextWords_df.filter(((F.col("review_text")) != ""))

# https://sparkbyexamples.com/pyspark/pyspark-where-filter/

In [ ]:
# review_df = review_df[(review_df["review_id"].isin(pd.unique(reviewTextWords_df["review_id"])))]

reviewTextWords_df_unique_review_ids = reviewTextWords_df.select(F.col("review_id")).distinct().collect()

review_df = review_df.filter(((F.col("review_text")).isin(reviewTextWords_df_unique_review_ids)))

In [ ]:

business_id_frequency = id_df.groupBy("business_id").count()
user_id_frequency = id_df.groupBy("user_id").count()

business_id_one_time = business_id_frequency.filter(((F.col("count"))==1)).select(F.col("business_id"))
user_id_one_time = user_id_frequency.filter(((F.col("count"))==1)).select(F.col("user_id"))

id_df = id_df.filter(~(
    ((F.col("business_id").isin(business_id_one_time))) & ((F.col("user_id").isin(user_id_one_time)))
))
# https://sparkbyexamples.com/pyspark/pyspark-groupby-count-explained/

In [ ]:

reviewTextWords_df = reviewTextWords_df.withColumn("word_count", F.lit(1))
# https://sparkbyexamples.com/pyspark/pyspark-add-new-column-to-dataframe/

In [ ]:

reviewTextWord2vec_df = reviewTextWords_df.join(word2vec_df, on="review_text")
# https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.join.html

# reviewTextWord2vec_df.sort_values(["review_id"], inplace=True)

In [ ]:


agg_column_names = vec_column_names + ["word_count"]

reviewTextWord2vec_df_agg = reviewTextWord2vec_df.groupBy("review_id").sum(agg_column_names)

# https://sparkbyexamples.com/pyspark/pyspark-groupby-agg-aggregate-explained/
# https://stackoverflow.com/questions/33882894/spark-sql-apply-aggregate-functions-to-a-list-of-columns

In [ ]:

id_review_df = id_df.join(reviewTextWord2vec_df_agg, on="review_id")

In [ ]:

id_review_df_agg_user = id_review_df.groupBy("user_id").sum(agg_column_names)

In [ ]:
id_review_df_agg_business = id_review_df.groupBy("business_id").sum(agg_column_names)

In [ ]:
user_vec_column_names = ["u"+x for x in vec_column_names]
business_vec_column_names = ["b"+x for x in vec_column_names]
for i in range(len(vec_column_names)):
    vec_column_name = vec_column_names[i]
    user_vec_column_name = user_vec_column_names[i]
    business_vec_column_name = business_vec_column_names[i]
    id_review_df_agg_user = id_review_df_agg_user.withColumnRenamed(vec_column_name, user_vec_column_name)
    id_review_df_agg_business = id_review_df_agg_business.withColumnRenamed(vec_column_name, business_vec_column_name)

id_review_df_agg_user = id_review_df_agg_user.withColumnRenamed("word_count", "user_word_count")
id_review_df_agg_business = id_review_df_agg_business.withColumnRenamed("word_count", "business_word_count")
# https://stackoverflow.com/questions/38798567/pyspark-rename-more-than-one-column-using-withcolumnrenamed

In [ ]:


id2vec_df = id2vec_df.join(id_review_df_agg_user, "user_id")
for i in range(len(vec_column_names)):
    vec_column_name = vec_column_names[i]
    user_vec_column_name = user_vec_column_names[i]
    id2vec_df = id2vec_df.withColumn(vec_column_name, 
                ((F.col(user_vec_column_name)) - (F.col(vec_column_name)))
                                    )
#https://www.datasciencemadesimple.com/sum-of-two-or-more-columns-in-pyspark/    
id2vec_df = id2vec_df.withColumn(vec_column_name, 
            ((F.col("user_word_count")) - (F.col("word_count")))
                                    )
id2vec_df = id2vec_df.filter(((F.col("user_word_count")) != 0))

for i in range(len(user_vec_column_names)):
    user_vec_column_name = user_vec_column_names[i]
    id2vec_df = id2vec_df.withColumn(user_vec_column_name, 
                                     ((F.col(user_vec_column_name)) / (F.col("user_word_count")))
                                    )
# https://mrpowers.medium.com/performing-operations-on-multiple-columns-in-a-pyspark-dataframe-36e97896c378

In [ ]:
id2vec_df = id2vec_df.join(id_review_df_agg_business, "business_id")
for i in range(len(vec_column_names)):
    vec_column_name = vec_column_names[i]
    business_vec_column_name = business_vec_column_names[i]
    id2vec_df = id2vec_df.withColumn(business_vec_column_name, 
                ((F.col(business_vec_column_name)) - (F.col(vec_column_name))))
#https://www.datasciencemadesimple.com/sum-of-two-or-more-columns-in-pyspark/    
id2vec_df = id2vec_df.withColumn(business_vec_column_name, 
            ((F.col("(business_word_count")) - (F.col("word_count"))))
id2vec_df = id2vec_df.filter(((F.col("(business_word_count")) != 0))

for i in range(len(business_vec_column_names)):
    business_vec_column_name = business_vec_column_names[i]
    id2vec_df = id2vec_df.withColumn(business_vec_column_name, 
                                     ((F.col(business_vec_column_name)) / (F.col("user_word_count"))))

# https://mrpowers.medium.com/performing-operations-on-multiple-columns-in-a-pyspark-dataframe-36e97896c378

In [ ]:
target_column_name = "target"
target_column_names = [target_column_name]
id2vec_df = id2vec_df.withColumn(target_column_name, F.lit(1))

concated_vec_column_names = user_vec_column_names + business_vec_column_names

data_column_names = concated_vec_column_names + target_column_names
data_column_names = [F.col(x) for x in data_column_names]
train_data_curr_excluded_from_average = id2vec_df.select(*data_column_names)

In [ ]:

for i in range(len(vec_column_names)):
    vec_column_name = vec_column_names[i]
    user_vec_column_name = user_vec_column_names[i]
    id_review_df_agg_user = id_review_df_agg_user.withColumn(user_vec_column_name, 
                                     ((F.col(vec_column_name)) / (F.col("word_count")))
                                    )
    #id_review_df_agg_user = id_review_df_agg_user.drop(F.col(vec_column_name))
    
    business_vec_column_name = business_vec_column_names[i]
    id_review_df_agg_business = id_review_df_agg_business.withColumn(business_vec_column_name, 
                                     ((F.col(vec_column_name)) / (F.col("word_count")))
                                    )
    #id_review_df_agg_business = id_review_df_agg_business.drop(F.col(vec_column_name))


valid_user_id_column = id2vec_df.select(F.col("user_id"))
valid_business_id_column = id2vec_df.select(F.col("business_id"))

In [ ]:
chunk_size = 5000
#len_set_valid_user_id_column = valid_user_id_column.distinct().count()
train_id_df = train_data_curr_excluded_from_average

seen_pairs = train_id_df.select(F.col("user_id"), F.col("business_id"))

user_fraction = np.ceil(chunk_size/valid_user_id_column.count())
business_fraction = np.ceil(chunk_size/valid_business_id_column.count())

temp_user_id_samples = valid_user_id_column.sample(fraction=1)
temp_business_id_samples = valid_business_id_column.sample(fraction=1)

temp_user_id_sampled = temp_user_id_samples.sample(fraction=user_fraction)
temp_user_id_sampled = temp_user_id_sampled.limit(chunk_size)
temp_business_id_sampled = temp_business_id_samples.sample(fraction=business_fraction)
temp_business_id_sampled = temp_business_id_sampled.limit(chunk_size)

temp_sampled_pairs = temp_user_id_sampled.withColumn("business_id", 
                                                       temp_business_id_sampled.select(F.col("business_id"))
                                                      )
unseen_pairs = temp_sampled_pairs.difference(seen_pairs)

len_train_df = train_id_df.count()
while unseen_pairs.count() < len_train_df:

    temp_user_id_samples = valid_user_id_column.sample(fraction=1)
    temp_business_id_samples = valid_business_id_column.sample(fraction=1)

    temp_user_id_sampled = temp_user_id_samples.sample(fraction=user_fraction)
    temp_user_id_sampled = temp_user_id_sampled.limit(chunk_size)
    temp_business_id_sampled = temp_business_id_samples.sample(fraction=business_fraction)
    temp_business_id_sampled = temp_business_id_sampled.limit(chunk_size)

    temp_sampled_pairs = temp_user_id_sampled.withColumn("business_id", 
                                                           temp_business_id_sampled.select(F.col("business_id"))
                                                          )
    temp_unseen_pairs = temp_sampled_pairs.difference(seen_pairs)
    unseen_pairs = unseen_pairs.union(temp_unseen_pairs)

seen_pairs_with_target = train_id_df.select(F.col("user_id"), F.col("business_id"), F.col(target_column_name))
unseen_pairs_with_target = unseen_pairs.withColumn(target_column_name, F.lit(0))
# https://www.datasciencemadesimple.com/set-difference-in-pyspark-difference-of-two-dataframe/

In [ ]:
final_interaction = seen_pairs_with_target.union(unseen_pairs_with_target)

final_interaction = final_interaction.sample(fraction=1)
train_interaction = train_interaction.limit(int(final_interaction.count() * 0.75))
test_interaction = final_interaction.subtract(train_interaction)
assert final_interaction.count() == (train_interaction.count() + test_interaction.count())

train_interaction = train_interaction.join(id_review_df_agg_user,"user_id").drop(F.col("user_word_count"))
train_interaction = train_interaction.join(id_review_df_agg_business,"business_id").drop(F.col("business_word_count"))

test_interaction = test_interaction.join(id_review_df_agg_user,"user_id").drop(F.col("user_word_count"))
test_interaction = test_interaction.join(id_review_df_agg_business,"business_id").drop(F.col("business_word_count"))

In [ ]:
final_interaction = final_interaction.join(id_review_df_agg_user,"user_id").drop(F.col("user_word_count"))
final_interaction = final_interaction.join(id_review_df_agg_business,"business_id").drop(F.col("business_word_count"))

In [ ]:
input_column_names = user_vec_column_names + business_vec_column_names
assembler = M.features.VectorAssembler(inputCols = input_column_names, outputCol='features')
final_interaction = assembler.transform(final_interaction).select(F.col('features'), F.col(target_column_name))

In [ ]:
splitted = final_interaction.randomSplit([0.75, 0.25], 1)
train_interaction = splitted[0]
test_interaction = splitted[1]

In [ ]:
input_column_names = user_vec_column_names + business_vec_column_names
input_column_names = [F.col(x) for x in input_column_names]
target_column_names = [target_column_name]
target_column_names = [F.col(x) for x in target_column_names]

train_X = train_interaction.select(*input_column_names)
train_y = train_interaction.select(*target_column_names)

test_X = test_interaction.select(*input_column_names)
test_y = test_interaction.select(*target_column_names)

In [ ]:
lr = M.classification.LogisticRegression(labelCol = target_column_name,
                                     featuresCol="features")
pl = M.Pipeline(stages=[dt])

In [ ]:
model = pl.fit(train_interaction)    
# https://medium.com/featurepreneur/logistic-regression-with-pyspark-in-10-steps-3edecd6fe3c5

In [ ]:
predictions = model.transform(test_X)

In [32]:
spark.stop()